In [30]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, f1_score ,accuracy_score

In [36]:
file = pd.read_csv('data/spam.csv', encoding = 'utf8')
#file.drop_duplicates(inplace=True)
#file.dropna(inplace=True)
file

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will ฬ_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [37]:
print('Ham :',file[file['v1']=='ham'].shape)
print('Spam :',file[file['v1']=='spam'].shape)

Ham : (4825, 5)
Spam : (747, 5)


In [38]:
arr_text=file.iloc[:]['v2'].values
arr_class=file.iloc[:]['v1'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(arr_text, arr_class, test_size=0.33, random_state=42)

In [39]:
vect = CountVectorizer()
vect.fit(arr_text)

CountVectorizer()

In [40]:
train_transformer_x = vect.transform(X_train)
detect_model = MultinomialNB().fit(train_transformer_x,y_train)

In [41]:
test_transformer_x = vect.transform(X_test)
y_pred = detect_model.predict(test_transformer_x)
print('F1 score =',f1_score(y_test, y_pred, average='macro'))
print('Accuracy =',accuracy_score(y_test, y_pred))

F1 score = 0.9561690266641244
Accuracy = 0.9793365959760739


In [42]:
from sklearn.model_selection import RandomizedSearchCV
parameters = {
 'alpha' : numpy.arange(0, 8, 0.01),
 'fit_prior' : [True,False]
}
rdsCV = RandomizedSearchCV(MultinomialNB(),param_distributions=parameters,cv=10)
rdsCV.fit(train_transformer_x,y_train)
print('Best Parameter = ',rdsCV.best_params_)

Best Parameter =  {'fit_prior': True, 'alpha': 0.88}


In [43]:
from sklearn.model_selection import GridSearchCV
parameters = {
 'alpha' : numpy.arange(0, 8, 0.01),
 'fit_prior' : [True,False]
}
rdsCV = RandomizedSearchCV(MultinomialNB(),param_distributions=parameters,cv=10)
rdsCV.fit(train_transformer_x,y_train)
print('Best Parameter = ',rdsCV.best_params_)

Best Parameter =  {'fit_prior': True, 'alpha': 0.89}


In [1]:
import glob
import codecs
import numpy
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score
import nltk
nltk.download('stopwords')

pd.set_option('display.max_rows', 20, 
              'display.max_columns', 100)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def manage_tariff(data):
#   data['length'] = data['heading'].apply(len)
  #  data['heading'] = data.heading.apply(range_txt)
    
    db = []
    for i in range(len(data)):
    
     
    
        db.append({'descript': data.product_desc[i] ,'class':data.heading[i] })
    
    return db

In [10]:
reads =pd.read_excel('data/master.xlsx',sheet_name='master')
#reads['tariff_code'] = reads['tariff_code'].map('{:06}'.format)
books = reads[['tariff_code','product_desc']]
books

,tariff_code,product_desc
0,010111,"Horses; live, pure-bred breeding animals"
1,010119,"Horses; live, other than pure-bred breeding an..."
2,010120,"Asses, mules and hinnies; live"
3,010210,"Bovine animals; live, pure-bred breeding animals"
4,010290,"Bovine animals; live, other than pure-bred bre..."
...,...,...
31043,970300,"Sculptures and statuary; original, in any mate..."
31044,970400,"Stamps, postage or revenue; stamp-postmarks, f..."
31045,970500,Collections and collectors' pieces; of zoologi...
31046,970600,Antiques; of an age exceeding one hundred years


In [12]:
len(books['tariff_code'].unique())

6554

In [14]:
# Author: Lars Buitinck
# License: BSD 3 clause
from collections import defaultdict
import re
import sys
from time import time

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import DictVectorizer, FeatureHasher


def n_nonzero_columns(X):
    """Returns the number of non-zero columns in a CSR matrix X."""
    return len(np.unique(X.nonzero()[1]))


def tokens(doc):
    """Extract tokens from doc.

    This uses a simple regex to break strings into tokens. For a more
    principled approach, see CountVectorizer or TfidfVectorizer.
    """
    return (tok.lower() for tok in re.findall(r"\w+", doc))


def token_freqs(doc):
    """Extract a dict mapping tokens from doc to their frequencies."""
    freq = defaultdict(int)
    for tok in tokens(doc):
        freq[tok] += 1
    return freq


categories = [
    'alt.atheism',
    'comp.graphics',
    'comp.sys.ibm.pc.hardware',
    'misc.forsale',
    'rec.autos',
    'sci.space',
    'talk.religion.misc',
]
# Uncomment the following line to use a larger set (11k+ documents)
# categories = None

print(__doc__)
print("Usage: %s [n_features_for_hashing]" % sys.argv[0])
print("    The default number of features is 2**18.")
print()

try:
    n_features = int(sys.argv[1])
except IndexError:
    n_features = 2 ** 18
except ValueError:
    print("not a valid number of features: %r" % sys.argv[1])
    sys.exit(1)


print("Loading 20 newsgroups training data")
raw_data, _ = fetch_20newsgroups(subset='train', categories=categories,
                                 return_X_y=True)
data_size_mb = sum(len(s.encode('utf-8')) for s in raw_data) / 1e6
print("%d documents - %0.3fMB" % (len(raw_data), data_size_mb))
print()

print("DictVectorizer")
t0 = time()
vectorizer = DictVectorizer()
vectorizer.fit_transform(token_freqs(d) for d in raw_data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_size_mb / duration))
print("Found %d unique terms" % len(vectorizer.get_feature_names()))
print()

print("FeatureHasher on frequency dicts")
t0 = time()
hasher = FeatureHasher(n_features=n_features)
X = hasher.transform(token_freqs(d) for d in raw_data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_size_mb / duration))
print("Found %d unique terms" % n_nonzero_columns(X))
print()

print("FeatureHasher on raw tokens")
t0 = time()
hasher = FeatureHasher(n_features=n_features, input_type="string")
X = hasher.transform(tokens(d) for d in raw_data)
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_size_mb / duration))
print("Found %d unique terms" % n_nonzero_columns(X))

Automatically created module for IPython interactive environment
Usage: D:\Anaconda\envs\pyviz-tutorial\lib\site-packages\ipykernel_launcher.py [n_features_for_hashing]
    The default number of features is 2**18.

not a valid number of features: '-f'


SystemExit: 1

In [18]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, f1_score ,accuracy_score

pd.set_option('display.max_rows', 20, 
              'display.max_columns', 100)

In [26]:
df = pd.read_excel('data/external.xlsx',sheet_name= 'Sheet1')
df

,product,code_12,code_8,code_6,code_4,code_2
0,THERMICAL SCREW 2D\r\n51207799AA,73181590,73181590,731815,7318,73
1,33682 ADJUSTABLE DELAY UNIT FOR MN,85364990,85364990,853649,8536,85
2,33671 undervoltage trip MN 200 to 250 V ...,85369099,85369099,853690,8536,85
3,GCR_NS630_1600C NS800N3PMF2.0E,85362019,85362019,853620,8536,85
4,EGX150-A Ethernet Gateway Link150,85369099,85369099,853690,8536,85
...,...,...,...,...,...,...
8759,"XA2ED21 SS,2 POSITION,1NO(SHORT)",85365099,85365099,853650,8536,85
8760,"XA2ED33 SS,3 POSITION,2NO(SHORT)",85365099,85365099,853650,8536,85
8761,HMIGTO6310A 12.1 Color Touch Panel SVGA...,85285910,85285910,852859,8528,85
8762,E223R_TR F-T WP SINGLE GANG SKT COVER (...,85366999,85366999,853669,8536,85


In [28]:
new = df["product"].str.split(" ", n = 1, expand = True) 
df["new_product"] = new[1].str.strip()
df["product_code"] = new[0].str.strip()
df

,product,code_12,code_8,code_6,code_4,code_2,new_product,product_code
0,THERMICAL SCREW 2D\r\n51207799AA,73181590,73181590,731815,7318,73,SCREW 2D\r\n51207799AA,THERMICAL
1,33682 ADJUSTABLE DELAY UNIT FOR MN,85364990,85364990,853649,8536,85,ADJUSTABLE DELAY UNIT FOR MN,33682
2,33671 undervoltage trip MN 200 to 250 V ...,85369099,85369099,853690,8536,85,undervoltage trip MN 200 to 250 V DC and,33671
3,GCR_NS630_1600C NS800N3PMF2.0E,85362019,85362019,853620,8536,85,NS800N3PMF2.0E,GCR_NS630_1600C
4,EGX150-A Ethernet Gateway Link150,85369099,85369099,853690,8536,85,Ethernet Gateway Link150,EGX150-A
...,...,...,...,...,...,...,...,...
8759,"XA2ED21 SS,2 POSITION,1NO(SHORT)",85365099,85365099,853650,8536,85,"SS,2 POSITION,1NO(SHORT)",XA2ED21
8760,"XA2ED33 SS,3 POSITION,2NO(SHORT)",85365099,85365099,853650,8536,85,"SS,3 POSITION,2NO(SHORT)",XA2ED33
8761,HMIGTO6310A 12.1 Color Touch Panel SVGA...,85285910,85285910,852859,8528,85,12.1 Color Touch Panel SVGA-TFT,HMIGTO6310A
8762,E223R_TR F-T WP SINGLE GANG SKT COVER (...,85366999,85366999,853669,8536,85,F-T WP SINGLE GANG SKT COVER (TRAN) IP55,E223R_TR


In [30]:
df.to_excel('data/temp_file.xlsx')
print('success')

success


## ค้นหาคำภาษาไทยที่น่าจะเป็นไปได้จากตัวอักษรขึ้นต้น

**แนวคิด** 
- นำพจนานุกรมคำภาษาไทยมาเก็บไว้ใน list ของ Python
- ลองตัวอักษรขึ้นต้นมาค้นหาจาก list คำศัพท์ภาษาไทยว่าพบตัวไหนที่มีตัวอักษรขึ้นต้นดังกล่าวเหมือนกัน แล้วส่ง list ออกมา

In [27]:
from pythainlp.corpus import thai_words
words=list(thai_words())

In [28]:
def find_word(word):
      return [i for i in words if i.startswith(word)]

In [29]:
find_word("แม")

['แมรีแลนด์',
 'แมชชีน',
 'แม่',
 'แม้แต่',
 'แมคโคร',
 'แม้น',
 'แมงดาทะเลหางกลม',
 'แมนจูเรีย',
 'แม่คู่',
 'แม่ไม้',
 'แมรี่แลนด์',
 'แม่สะเรียง',
 'แมลงวันผลไม้',
 'แมลงหวี่',
 'แมริแลนด์',
 'แม่พระคงคา',
 'แม็ตต์',
 'แมะ',
 'แมวน้ำ',
 'แมวมอง',
 'แมงอ่อนช้อย',
 'แมงหางแดง',
 'แมงกานิน',
 'แม่เรือน',
 'แมส',
 'แมมโมแกรม',
 'แม่นาง',
 'แมสซาชูเซ็ทส์',
 'แมมบ้า',
 'แม่ยั่วเมือง',
 'แม่ธรณี',
 'แมลลิค',
 'แม่ยาย',
 'แม็ก',
 'แมลง',
 'แม่ตาย',
 'แมกไซไซ',
 'แม่ทัพ',
 'แมกนีเซียม',
 'แมลบ',
 'แมงปากจอบ',
 'แม่ฝา',
 'แมลงดา',
 'แม่ย้าว',
 'แมวเซา',
 'แม่ลาย',
 'แม่คุณ',
 'แมลงมุม',
 'แม่กระชังก้นรั่ว',
 'แมงไฟเดือนห้า',
 'แม่ย่างม้าเหาะ',
 'แม่งาน',
 'แม่พันธุ์',
 'แมงกะพรุน',
 'แม่อยู่หัว',
 'แมรี่',
 'แม่งม้าง',
 'แมนฯยู',
 'แมลงดาขี้ควาย',
 'แม่ออน',
 'แมชีนเนอรี่',
 'แม่วี',
 'แม่สายบัวแต่งตัวค้าง',
 'แม้กระนั้น',
 'แมคโครไบโอติก',
 'แม่หนัก',
 'แมนยูฯ',
 'แมลงปีกแข็ง',
 'แม่ม่ายทรงเครื่อง',
 'แม่น',
 'แม่ย่านาง',
 'แมลงสีเสียด',
 'แมคเคอเรล',
 'แม็กซ์',
 'แมคนีเซียม',
 'แม่ปะ',
 'แม

In [30]:
find_word("มะม่วง")

['มะม่วงหิมพานต์', 'มะม่วงป่า', 'มะม่วง']

Spellchecker
===

การรับ Input จาก User สิ่งที่เราจะได้พบเจออยู่ตลอด คือ User กรอกข้อมูลผิด ยิ่งข้อมูลที่ไม่ใช่ข้อมูลที่มีโครงสร้าง Structure Data, ไม่มี Data Type ตัวเลข หรือวันเวลา ที่มีรูปแบบแน่นอน แต่เป็นข้อความ Free Text ที่มีคำสะกดผิดปนอยู่ ทำให้การ Validate ข้อมูล อาจจะทำได้ยาก

Spell checker คือ โปรแกรมตรวจการสะกด ตรวจคำผิด ว่าข้อความ คำที่ User กรอกเข้ามามีปรากฎอยู่ใน Dictionary หรือไม่ โดยอาจจะแนะนำคำใกล้เคียง ที่น่าจะเป็นคำที่ถูกต้องให้ User เลือก หรือแม้กระทั่งเลือกให้โดยอัตโนมัติ เรียกว่า Spelling Correction

ในเคสของภาษาไทย โดย Default แล้ว Spellchecker ของ PyThaiNLP จะใช้อัลกอริทึม ของ Peter Norvig ที่จะหารายการคำใกล้เคียงจาก Dictionary โดยใช้จำนวนอักษรที่ผิด 1, 2, … ตัวอักษร ผสมกับความน่าจะเป็น จากความถี่ของคำนั้นที่ปรากฎใน Corpus 

In [31]:
# Install

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [32]:
# ปิด Warning ไว้ก่อน จะได้อ่านผลลัพท์ง่าย

import warnings
warnings.filterwarnings('ignore')

Spellchecker โปรแกรมตรวจตัวสะกด
โดย Default แล้ว Spellchecker ของ PyThaiNLP จะใช้อัลกอริทึม ของ Peter Norvig ที่จะหารายการคำใกล้เคียงจาก Dictionary โดยใช้จำนวนอักษรที่ผิด 1, 2, ... ตัวอักษร ผสมกับความน่าจะเป็น จากความถี่ของคำนั้นที่ปรากฎใน Corpus

โดยไม่ได้ใช้ Context หรือคำแวดล้อมที่มาก่อนหน้า หรือต่อจากนั้น และไม่ได้ใช้ตำแหน่งปุ่มใกล้เคียง บน Keyboard มาคำนวนความน่าจะเป็น

In [33]:
import pythainlp

pythainlp.__version__

'2.2.2'

In [34]:
from pythainlp import spell

In [35]:
spell("รั๊ว") # รัว ไม้ตรี

['รั้ว', 'รั่ว', 'อั๊ว', 'รัว', 'ปั๊ว', 'จั๊ว', 'ตั๊ว', 'กั๊ว', 'ลั๊ว']

In [36]:
spell("กวาา") 

['กว่า', 'กวาด', 'กวาง', 'กวา', 'กวาน', 'กวาว', 'กว้า']

In [37]:
spell("น้ำสม")

['น้ำส้ม', 'น้ำนม']

In [38]:
spell("ศาธารณสุบ")


['สาธารณสุข']

In [39]:
#ตัวเลขปน

spell("ร้อ4")

['ร้อน', 'ร้อง', 'ร้อย', 'ร้อก', 'ร้อ', 'ร้อด', 'ร้อบ', 'ร้อม']

In [40]:
#ปน ภาษาอังกฤษ ตัว o โอ

spell("รั้o")

['รั้ว', 'รั้ง', 'รั้น']

In [41]:
# ปน ภาษาอังกฤษ ตัว h

spell("ชhอน")

['ช้อน', 'ชอน', 'ช่อน']

## Spellchecker ด้วย Custom Dictionary และ Word Frequency

โดย Default จะใช้ Dictionary จาก Thai National Corpus (TNC) ซึ่งจะมีคำศัพท์ ใน Corpus จำนวนเกือบ 4 หมื่นคำ (Default)

In [11]:
from pythainlp.spell import NorvigSpellChecker

In [12]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
len(checker.dictionary())

39963

In [13]:
# ดูตัวอย่าง คำศัพท์ และ ความถี่ ที่อยู่ใน Dictionary

list(checker.dictionary())[:20]

[('ราชสกุล', 69),
 ('เภ็ก', 4),
 ('ญึด', 6),
 ('กบ่าว', 4),
 ('ปริตร', 3),
 ('กระหวัด', 49),
 ('อวัสดา', 3),
 ('เยี่ยง', 359),
 ('ลื้อ', 789),
 ('เพะ', 5),
 ('ภูษิต', 4),
 ('ซาวเสียง', 5),
 ('มกา', 4),
 ('ฟลูออเรสเซนซ์', 2),
 ('อินดิก', 2),
 ('ระเหระหน', 6),
 ('เอ็กซ์.', 5),
 ('มกิจจ์', 3),
 ('แพลงก์ตอน', 142),
 ('บุ๋ง', 8)]